In [1]:
import cv2  #image proccessing
import os, glob #for directory operations
import numpy as np
from moviepy.editor import VideoFileClip  #for reading & writing videos

In [2]:
#for convertion RGB2HLS (as it is difficult to detect yellow lanes in RGB )
def convert_hls(image):
    return cv2.cvtColor(image, cv2.COLOR_RGB2HLS)

In [3]:
#fn for applying mask to convert the RGB img to pixals
def select_white_yellow(image): # as the lanes are always yellow or white
    converted = convert_hls(image)
    # white color mask
    lower_white = np.uint8([  0, 200,   0])
    upper_white = np.uint8([255, 255, 255])
    white_mask = cv2.inRange(converted, lower_white, upper_white)
    # yellow color mask
    lower_yellow = np.uint8([ 10,   0, 100])
    upper_yellow = np.uint8([ 40, 255, 255])
    yellow_mask = cv2.inRange(converted, lower_yellow, upper_yellow)
    # combine the mask
    mask = cv2.bitwise_or(white_mask, yellow_mask)
    return cv2.bitwise_and(image, image, mask = mask)

In [4]:
#for Canny edge detection
#convertion of image to grayscle to find edges (measure chane in pixels)
def convert_gray_scale(image):
    return cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

In [5]:
#fn for removing noise from canny edge
def apply_smoothing(image, kernel_size=15):
    return cv2.GaussianBlur(image, (kernel_size, kernel_size), 0)

In [6]:
#detecting edges as Canny edge detection uses the fact that edges has high gradients
def detect_edges(image, low_threshold=50, high_threshold=150):
    return cv2.Canny(image, low_threshold, high_threshold)

In [7]:
#Selection of region of interest (ROI)
def filter_region(image, vertices):
   
    #Create the mask using the vertices and apply it to the input image
    
    mask = np.zeros_like(image)
    if len(mask.shape)==2:
        cv2.fillPoly(mask, vertices, 255)
    else:
        cv2.fillPoly(mask, vertices, (255,)*mask.shape[2]) # in case, the input image has a channel dimension        
    return cv2.bitwise_and(image, mask)
    
def select_region(image):
  
    #keeps the region surrounded by the vertices (polygon), Other area is set to 0 (black).
  
    # first, define the polygon by vertices
    rows, cols = image.shape[:2]
    bottom_left  = [cols*0.1, rows*0.95]
    top_left     = [cols*0.4, rows*0.6]
    bottom_right = [cols*0.9, rows*0.95]
    top_right    = [cols*0.6, rows*0.6] 
    
    # the vertices are an array of polygons ( array of arrays) and the data type must be integer
    vertices = np.array([[bottom_left, top_left, top_right, bottom_right]], dtype=np.int32)
    return filter_region(image, vertices)

In [8]:
#Hough transformation (detects any shape can be represented mathematically)
def hough_lines(image):
    #image should be the output of a Canny transform.
    #Returns hough lines (not the image with lines)
    return cv2.HoughLinesP(image, rho=1, theta=np.pi/180, threshold=20, minLineLength=20, maxLineGap=300)

In [9]:
#getting the average line for multiple lines that can be detected for a lane line
#then extrapolate the line to cover full lane line length for partially recognized lane lines
def average_slope_intercept(lines):
    left_lines    = [] # (slope, intercept)
    left_weights  = [] # (length,)
    right_lines   = [] # (slope, intercept)
    right_weights = [] # (length,)
    
    for line in lines:
        for x1, y1, x2, y2 in line:
            if x2==x1:
                continue # ignore a vertical line
            slope = (y2-y1)/(x2-x1)
            intercept = y1 - slope*x1
            length = np.sqrt((y2-y1)**2+(x2-x1)**2)
            if slope < 0: # y is reversed in image
                left_lines.append((slope, intercept))
                left_weights.append((length))
            else:
                right_lines.append((slope, intercept))
                right_weights.append((length))
    
    # add more weight to longer lines    
    left_lane  = np.dot(left_weights,  left_lines) /np.sum(left_weights)  if len(left_weights) >0 else None
    right_lane = np.dot(right_weights, right_lines)/np.sum(right_weights) if len(right_weights)>0 else None
    
    return left_lane, right_lane # (slope, intercept), (slope, intercept)

In [10]:
#drawing lanes
def make_line_points(y1, y2, line):
    #Convert a line represented in slope and intercept into pixel points  
    if line is None:
        return None    
    slope, intercept = line    
    # make sure everything is integer as cv2.line requires it
    x1 = int((y1 - intercept)/slope)
    x2 = int((y2 - intercept)/slope)
    y1 = int(y1)
    y2 = int(y2)
    return ((x1, y1), (x2, y2))

In [11]:
def lane_lines(image, lines):
    left_lane, right_lane = average_slope_intercept(lines)
    
    y1 = image.shape[0] # bottom of the image
    y2 = y1*0.6         # slightly lower than the middle
    left_line  = make_line_points(y1, y2, left_lane)
    right_line = make_line_points(y1, y2, right_lane)
    
    return left_line, right_line

In [12]:
def draw_lane_lines(image, lines, color=[255, 0, 0], thickness=20):
    # make a separate image to draw lines and combine with the orignal later
    line_image = np.zeros_like(image)
    for line in lines:
        if line is not None:
            cv2.line(line_image, *line,  color, thickness)
    # image1 * α + image2 * β + λ
    # image1 and image2 must be the same shape.
    return cv2.addWeighted(image, 1.0, line_image, 0.95, 0.0)

In [13]:
# Algo for detecting lanes and saving output
from collections import deque
QUEUE_LENGTH=50
class LaneDetector:
    def __init__(self):
        self.left_lines  = deque(maxlen=QUEUE_LENGTH)
        self.right_lines = deque(maxlen=QUEUE_LENGTH)
    def process(self, image):
        white_yellow = select_white_yellow(image)
        gray         = convert_gray_scale(white_yellow)
        smooth_gray  = apply_smoothing(gray)
        edges        = detect_edges(smooth_gray)
        regions      = select_region(edges)
        lines        = hough_lines(regions)
        left_line, right_line = lane_lines(image, lines)
        def mean_line(line, lines):
            if line is not None:
                lines.append(line)
            if len(lines)>0:
                line = np.mean(lines, axis=0, dtype=np.int32)
                line = tuple(map(tuple, line)) # make sure it's tuples not numpy array for cv2.line to work
            return line
        left_line  = mean_line(left_line,  self.left_lines)
        right_line = mean_line(right_line, self.right_lines)
        return draw_lane_lines(image, (left_line, right_line))

In [14]:
#fn for the input video and its conversion to the output
def process_video(video_input, video_output):
    detector = LaneDetector()

    clip = VideoFileClip(os.path.join('test_videos', video_input))
    processed = clip.fl_image(detector.process)
    processed.write_videofile(os.path.join('output_videos', video_output), audio=False)

In [16]:
process_video(r'C:\Users\DELL\Downloads\Lane Detection Task\Data\Videos\video1.mp4',r'E:\Lane Detection Task\Data\output video\video1op.mp4')

t:   0%|                                                                             | 0/485 [00:00<?, ?it/s, now=None]

Moviepy - Building video E:\Lane Detection Task\Data\output video\video3op.mp4.
Moviepy - Writing video E:\Lane Detection Task\Data\output video\video3op.mp4




t:  27%|██████████████████▏                                                | 132/485 [00:08<00:17, 20.68it/s, now=None]

TypeError: 'NoneType' object is not iterable


t:  28%|██████████████████▌                                                | 134/485 [00:23<00:16, 20.68it/s, now=None]

In [ ]:
process_video(r'C:\Users\DELL\Downloads\Lane Detection Task\Data\Videos\video2.mp4',r'E:\Lane Detection Task\Data\output video\video2op.mp4')

t:  28%|██████████████████▌                                                | 134/485 [00:20<00:17, 19.73it/s, now=None]